In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data
df=pd.read_csv('InternetAds_norm_19.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,...,att1550,att1551,att1552,att1553,att1554,att1555,att1556,att1557,att1558,outlier
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [3]:
df.shape

(3264, 1556)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/InternetAds.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,1555]
X = df[:,0:1555]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/kader/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
          # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-05-15 10:21:47,617][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1555)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1555)],y_test.shape=(1078,)
[ 2018-05-15 10:21:47,647][cascade_classifier.fit_transform] group_dims=[1555]
[ 2018-05-15 10:21:47,648][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:21:47,649][cascade_classifier.fit_transform] group_ends=[1555]
[ 2018-05-15 10:21:47,650][cascade_classifier.fit_transform] X_train.shape=(2186, 1555),X_test.shape=(1078, 1555)
[ 2018-05-15 10:21:47,679][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1555), X_cur_test.shape=(1078, 1555)
[ 2018-05-15 10:21:50,819][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-05-15 10:21:54,491][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.35%
[ 2018-05-15 10:21:58,268][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 10:22:36,933][cascade_classifier.transform] X_groups_test.shape=[(1078, 1555)]
[ 2018-05-15 10:22:36,937][cascade_classifier.transform] group_dims=[1555]
[ 2018-05-15 10:22:36,938][cascade_classifier.transform] X_test.shape=(1078, 1555)
[ 2018-05-15 10:22:36,941][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1555)


Test Accuracy of ExtraTrees = 97.031540 %


In [10]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[903  10]
 [ 22 143]]


             precision    recall  f1-score   support

        0.0       0.98      0.99      0.98       913
        1.0       0.93      0.87      0.90       165

avg / total       0.97      0.97      0.97      1078



In [11]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-05-15 10:22:38,718][cascade_classifier.transform] X_groups_test.shape=[(1078, 1555)]
[ 2018-05-15 10:22:38,725][cascade_classifier.transform] group_dims=[1555]
[ 2018-05-15 10:22:38,727][cascade_classifier.transform] X_test.shape=(1078, 1555)
[ 2018-05-15 10:22:38,733][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1555)


In [12]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-05-15 10:22:40,048][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1557)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1557)],y_test.shape=(1078,)
[ 2018-05-15 10:22:40,073][cascade_classifier.fit_transform] group_dims=[1557]
[ 2018-05-15 10:22:40,074][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:22:40,077][cascade_classifier.fit_transform] group_ends=[1557]
[ 2018-05-15 10:22:40,080][cascade_classifier.fit_transform] X_train.shape=(2186, 1557),X_test.shape=(1078, 1557)
[ 2018-05-15 10:22:40,108][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1557), X_cur_test.shape=(1078, 1557)
[ 2018-05-15 10:22:42,986][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.72%
[ 2018-05-15 10:22:45,830][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-05-15 10:22:48,792][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [13]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 10:23:12,489][cascade_classifier.transform] X_groups_test.shape=[(1078, 1557)]
[ 2018-05-15 10:23:12,501][cascade_classifier.transform] group_dims=[1557]
[ 2018-05-15 10:23:12,506][cascade_classifier.transform] X_test.shape=(1078, 1557)
[ 2018-05-15 10:23:12,516][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1557)


Test Accuracy of ExtraTrees = 96.567718 %


In [14]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[903  10]
 [ 27 138]]


             precision    recall  f1-score   support

        0.0       0.97      0.99      0.98       913
        1.0       0.93      0.84      0.88       165

avg / total       0.97      0.97      0.96      1078



In [15]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')
    
    # Matrix de confusion
    from sklearn.metrics import classification_report, confusion_matrix
    print(confusion_matrix(y_test,y_pred))
    print('\n')
    print(classification_report(y_test,y_pred))

In [16]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-05-15 10:23:14,139][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1557)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1557)],y_test.shape=(1078,)
[ 2018-05-15 10:23:14,147][cascade_classifier.fit_transform] group_dims=[1557]
[ 2018-05-15 10:23:14,149][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 10:23:14,150][cascade_classifier.fit_transform] group_ends=[1557]
[ 2018-05-15 10:23:14,152][cascade_classifier.fit_transform] X_train.shape=(2186, 1557),X_test.shape=(1078, 1557)
[ 2018-05-15 10:23:14,171][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1557), X_cur_test.shape=(1078, 1557)
[ 2018-05-15 10:23:17,234][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-05-15 10:23:20,238][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-05-15 10:23:23,324][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of ExtraTrees = 96.846011 %', 0, 'fois')
[[905   8]
 [ 26 139]]


             precision    recall  f1-score   support

        0.0       0.97      0.99      0.98       913
        1.0       0.95      0.84      0.89       165

avg / total       0.97      0.97      0.97      1078



[ 2018-05-15 10:23:49,769][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-05-15 10:23:52,690][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-05-15 10:23:55,854][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=95.89%
[ 2018-05-15 10:23:59,096][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.72%
[ 2018-05-15 10:24:02,360][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.72%
[ 2018-05-15 10:24:05,551][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.17%
[ 2018-05-15 10:24:08,748][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=96.35%
[ 2018-05-15 10:24:11,819][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=96.79%


('Test Accuracy of ExtraTrees = 95.732839 %', 1, 'fois')
[[895  18]
 [ 28 137]]


             precision    recall  f1-score   support

        0.0       0.97      0.98      0.97       913
        1.0       0.88      0.83      0.86       165

avg / total       0.96      0.96      0.96      1078



[ 2018-05-15 10:24:22,675][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-05-15 10:24:25,760][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-05-15 10:24:28,811][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.35%
[ 2018-05-15 10:24:31,939][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.17%
[ 2018-05-15 10:24:34,998][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.72%
[ 2018-05-15 10:24:38,114][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.63%
[ 2018-05-15 10:24:41,203][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.26%
[ 2018-05-15 10:24:44,271][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.71%


...

('Test Accuracy of ExtraTrees = 96.938776 %', 17, 'fois')
[[904   9]
 [ 24 141]]


             precision    recall  f1-score   support

        0.0       0.97      0.99      0.98       913
        1.0       0.94      0.85      0.90       165

avg / total       0.97      0.97      0.97      1078



[ 2018-05-15 10:32:17,562][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-05-15 10:32:20,149][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.72%
[ 2018-05-15 10:32:22,802][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.35%
[ 2018-05-15 10:32:25,556][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.72%
[ 2018-05-15 10:32:28,264][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.26%
[ 2018-05-15 10:32:30,999][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.17%
[ 2018-05-15 10:32:33,504][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.26%
[ 2018-05-15 10:32:36,117][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.25%


('Test Accuracy of ExtraTrees = 96.753247 %', 18, 'fois')
[[906   7]
 [ 28 137]]


             precision    recall  f1-score   support

        0.0       0.97      0.99      0.98       913
        1.0       0.95      0.83      0.89       165

avg / total       0.97      0.97      0.97      1078



[ 2018-05-15 10:32:45,559][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-05-15 10:32:48,101][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.26%
[ 2018-05-15 10:32:50,727][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.80%
[ 2018-05-15 10:32:53,483][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.72%
[ 2018-05-15 10:32:56,230][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.26%
[ 2018-05-15 10:32:58,891][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.63%
[ 2018-05-15 10:33:01,523][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=96.35%
[ 2018-05-15 10:33:04,121][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.25%


('Test Accuracy of ExtraTrees = 96.753247 %', 19, 'fois')
[[904   9]
 [ 26 139]]


             precision    recall  f1-score   support

        0.0       0.97      0.99      0.98       913
        1.0       0.94      0.84      0.89       165

avg / total       0.97      0.97      0.97      1078



# ('Test Accuracy of ExtraTrees = 97.124304 %', 3, 'fois')